### 删除集合：


# AnalyticDB

[AnalyticDB for PostgreSQL](https://www.alibabacloud.com/help/en/analyticdb-for-postgresql/product-overview/overview-product-overview)是一个大规模并行处理（MPP）数据仓库服务，旨在在线分析大量数据。

要运行此笔记本，您需要在云中运行一个AnalyticDB for PostgreSQL实例（您可以在[common-buy.aliyun.com](https://common-buy.aliyun.com/?commodityCode=GreenplumPost&regionId=cn-hangzhou&request=%7B%22instance_rs_type%22%3A%22ecs%22%2C%22engine_version%22%3A%226.0%22%2C%22seg_node_num%22%3A%224%22%2C%22SampleData%22%3A%22false%22%2C%22vector_optimizor%22%3A%22Y%22%7D)获取一个实例)。

创建实例后，您应该通过[API](https://www.alibabacloud.com/help/en/analyticdb-for-postgresql/developer-reference/api-gpdb-2016-05-03-createaccount)或在实例详细信息网页的“帐号管理”中创建一个管理账户。

您应该确保已安装`llama-index`：


In [ ]:
%pip install llama-index-vector-stores-analyticdb

In [ ]:
!pip install llama-index

### 请提供参数：


In [ ]:
import osimport getpass# 阿里云RAM的AK和SK：alibaba_cloud_ak = ""alibaba_cloud_sk = ""# 实例信息：region_id = "cn-hangzhou"  # 特定实例的区域IDinstance_id = "gp-xxxx"  # ADB实例IDaccount = "test_account"  # 由API创建或在实例详细页面的“帐号管理”中创建的实例帐号名称account_password = ""  # 实例帐号密码

### 导入所需的包依赖项：


In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.vector_stores.analyticdb import AnalyticDBVectorStore

# AnalyticDB

[AnalyticDB for PostgreSQL](https://www.alibabacloud.com/help/en/analyticdb-for-postgresql/product-overview/overview-product-overview)是一个大规模并行处理（MPP）数据仓库服务，旨在在线分析大量数据。

要运行此笔记本，您需要在云中运行一个AnalyticDB for PostgreSQL实例（您可以在[common-buy.aliyun.com](https://common-buy.aliyun.com/?commodityCode=GreenplumPost&regionId=cn-hangzhou&request=%7B%22instance_rs_type%22%3A%22ecs%22%2C%22engine_version%22%3A%226.0%22%2C%22seg_node_num%22%3A%224%22%2C%22SampleData%22%3A%22false%22%2C%22vector_optimizor%22%3A%22Y%22%7D)获取一个实例)。

创建实例后，您应该通过[API](https://www.alibabacloud.com/help/en/analyticdb-for-postgresql/developer-reference/api-gpdb-2016-05-03-createaccount)或在实例详细信息网页的“帐号管理”中创建一个管理账户。

您应该确保已安装`llama-index`：


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

### 读取数据：


In [ ]:
# 加载文档documents = SimpleDirectoryReader("./data/paul_graham/").load_data()print(f"总文档数：{len(documents)}")print(f"第一篇文档，id：{documents[0].doc_id}")print(f"第一篇文档，哈希值：{documents[0].hash}")print(    "第一篇文档，文本"    f"（{len(documents[0].text)} 个字符）：\n{'='*20}\n{documents[0].text[:360]} ...")

### 创建 AnalyticDB Vector 存储对象：


In [ ]:
analytic_db_store = AnalyticDBVectorStore.from_params(
    access_key_id=alibaba_cloud_ak,
    access_key_secret=alibaba_cloud_sk,
    region_id=region_id,
    instance_id=instance_id,
    account=account,
    account_password=account_password,
    namespace="llama",
    collection="llama",
    metrics="cosine",
    embedding_dimension=1536,
)

### 从文档中构建索引：


In [ ]:
storage_context = StorageContext.from_defaults(vector_store=analytic_db_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

### 使用索引进行查询：


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Why did the author choose to work on AI?")

print(response.response)

### 删除集合：


In [ ]:
analytic_db_store.delete_collection()